# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that UnicornFlix has trained models for 3 different use cases (Top Picks for You, More like X and Reranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Batch Recommendations](#batch)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data.ipynb`](01_Data.ipynb), and [`02_Training.ipynb`](02_Training.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 


In this Notebook we will interact with Recommenders and Campaigns and get recommendations. 

We will create and interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. with the section we are building  in this notebook highlighted in blue with a dashed outline.

![Workflow](images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/imdb/items.csv', index_col=0)

# Render some sample data
items_df.head(5)

,TITLE,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,PLOT,US_MATURITY_RATING_STRING,US_MATURITY_RATING,GENRES,CREATION_TIMESTAMP,PROMOTION
ITEM_ID,,,,,,,,,,
tt0906325,Bagboy,2007,4,741,A teenager enters the competitive world of gro...,PG-13,13,Comedy,1167609600,False
tt0906665,Sukiyaki Western Django,2007,6,14582,A nameless gunfighter arrives in a town ripped...,R,17,Action|Western,1167609600,False
tt0907657,Once,2007,8,109787,A modern-day musical about a busker and an imm...,R,17,Drama|Music|Romance,1167609600,False
tt0910905,In the Electric Mist,2009,6,16592,A detective in post-Katrina New Orleans has a ...,R,17,Crime|Drama|Mystery|Thriller,1230768000,False
tt0910554,Frequently Asked Questions About Time Travel,2009,7,32051,"While drinking at their local pub, three socia...",PG-13,13,Comedy|Sci-Fi,1230768000,False


By defining the ID column as the index column it is easy to return a movie by just querying the ID. 

In [5]:
movie_id_example = 'tt0095016'
title = items_df.loc[movie_id_example]['TITLE']
print(title)

Die Hard


## Interact with recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "More like X" Recommender

'More like X' requires an item and a user as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from the "More like X" Recommender and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Die Hard).

We will be using the `recommenderArn`, the `itemId`, the `userId` as well as the number or results we want, `numResults`.

In [6]:
# First pick a user
test_user_id = "1"

Let's use the capability of Amazon Personalize to return item metadata to get the movie title.

In [7]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_more_like_x_arn,
    itemId = movie_id_example,
    userId = test_user_id,
    numResults = 20,
    metadataColumns = {
            "ITEMS": ['TITLE']
        }
)

First lets get the response directly from the get_recommendations API, which by default returns 25 items, but can be adjusted.

In [8]:
print(get_recommendations_response)

{'ResponseMetadata': {'RequestId': 'a1053daa-a96a-44b7-8570-9832c798ba1c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 27 Apr 2024 23:29:49 GMT', 'content-type': 'application/json', 'content-length': '2757', 'connection': 'keep-alive', 'x-amzn-requestid': 'a1053daa-a96a-44b7-8570-9832c798ba1c'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': 'tt0103064', 'metadata': {'title': 'Terminator 2: Judgment Day'}}, {'itemId': 'tt0090605', 'metadata': {'title': 'Aliens'}}, {'itemId': 'tt0086190', 'metadata': {'title': 'Star Wars: Episode VI - Return of the Jedi'}}, {'itemId': 'tt0078748', 'metadata': {'title': 'Alien'}}, {'itemId': 'tt0112864', 'metadata': {'title': 'Die Hard with a Vengeance'}}, {'itemId': 'tt0099810', 'metadata': {'title': 'The Hunt for Red October'}}, {'itemId': 'tt0099423', 'metadata': {'title': 'Die Hard 2'}}, {'itemId': 'tt0114746', 'metadata': {'title': '12 Monkeys'}}, {'itemId': 'tt0120382', 'metadata': {'title': 'The Truman Show'}}, {'itemId': 'tt0080684', 'meta

Depending on your Item catalog, this response will not be easy to quickly evaluate the recommendations, so lets use the function we created above.

In [9]:
item_list = get_recommendations_response['itemList']
print("If you liked " + items_df.loc[movie_id_example]['TITLE'] + ", you may also like:")
print()
for item in item_list:
    print(item['metadata']['title'])

If you liked Die Hard, you may also like:

Terminator 2: Judgment Day
Aliens
Star Wars: Episode VI - Return of the Jedi
Alien
Die Hard with a Vengeance
The Hunt for Red October
Die Hard 2
12 Monkeys
The Truman Show
Star Wars: Episode V - The Empire Strikes Back
Full Metal Jacket
2001: A Space Odyssey
Speed
The Lion King
Dead Poets Society
The Dark Knight
The Good, the Bad and the Ugly
Interview with the Vampire: The Vampire Chronicles
Shrek
The Princess Bride


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar movies render in a nice dataframe. Again, let's create a helper function to achieve this.

In [10]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_id, user_id):
    # Get the movie name
    movie_name = items_df.loc[movie_id_example]['TITLE']
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_more_like_x_arn,
        itemId = str(movie_id),
        userId = user_id,
        numResults = 15,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Builda new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        try:
            movie = item['metadata']['title']
            recommendation_list.append(movie)
        except:
            pass
    new_rec_df = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Let's sample some data from our dataset to test our "More like X" Recommender. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [11]:
samples = items_df.sample(5)
samples

,TITLE,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,PLOT,US_MATURITY_RATING_STRING,US_MATURITY_RATING,GENRES,CREATION_TIMESTAMP,PROMOTION
ITEM_ID,,,,,,,,,,
tt0105316,Sarafina!,1992,6,1805,South African teenagers fight against aparthei...,R,17,Drama|Musical,694224000,False
tt0138304,The Astronaut's Wife,1999,5,52483,After an explosion in space and subsequent two...,R,17,Drama|Sci-Fi|Thriller,915148800,False
tt0091055,Firewalker,1986,5,5923,A pair of adventurers try to track down an anc...,PG,8,Action|Adventure|Comedy,504921600,False
tt1185616,Waltz with Bashir,2008,8,54655,An Israeli film director interviews fellow vet...,R,17,Documentary|Animation|Biography|Drama|History|...,1199145600,False
tt0062467,Wait Until Dark,1967,8,27412,A recently blinded woman is terrorized by a tr...,Approved,8,Thriller,-94694400,False


In [12]:
more_like_x_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    more_like_x_recommendations_df = get_new_recommendations_df(more_like_x_recommendations_df, movie, test_user_id)

more_like_x_recommendations_df

,Die Hard,Die Hard,Die Hard,Die Hard,Die Hard
0,Forrest Gump,American Pie,Escape from L.A.,Eternal Sunshine of the Spotless Mind,High Noon
1,Four Weddings and a Funeral,Bowfinger,Coneheads,The Pursuit of Happyness,High Sierra
2,The Bodyguard,Arlington Road,Star Wars: Episode IV - A New Hope,Forrest Gump,Better Luck Tomorrow
3,Sleepless in Seattle,School of Rock,Superman IV: The Quest for Peace,The Blind Side,Star Wars: Episode IV - A New Hope
4,Pretty Woman,Star Wars: Episode I - The Phantom Menace,101 Dalmatians,The King's Speech,2001: A Space Odyssey
5,Die Hard 2,Gladiator,Superman III,Gran Torino,Beauty and the Beast
6,Batman Forever,American Psycho,Star Trek: The Motion Picture,12 Angry Men,Planet of the Apes
7,Mrs. Doubtfire,Spider-Man,2001: A Space Odyssey,Million Dollar Baby,The Fifth Element
8,True Lies,The Sixth Sense,Planet of the Apes,Thank You for Smoking,Run Lola Run
9,Speed,American Beauty,Star Trek: Insurrection,American Beauty,Willy Wonka & the Chocolate Factory


You may notice that some of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). Inside our fictional streaming service UnicornFlix we can use this recommender to provide recommendations within a movie detail page with the heading "If you like this movie, you may also like".

### "Top picks for you" Recommender

"Top picks for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since "Top picks for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [13]:
users = random.sample(range(1, 600), 3)

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Top picks for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a movie.

The cells below will handle getting recommendations from the "Top picks for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [14]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):

    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = 15,
        metadataColumns = {
                "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        try:
            movie = item['metadata']['title']
        except:
            movie = items_df.loc[item['itemId']]['TITLE']
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [15]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,278,583,479
0,Up,Interstellar,Star Trek: The Motion Picture
1,Apollo 13,Ant-Man,Moonraker
2,Harry Potter and the Half-Blood Prince,The Imitation Game,Conan the Destroyer
3,The Incredibles,The Martian,The Karate Kid Part II
4,Toy Story,Guardians of the Galaxy,Star Trek V: The Final Frontier
5,Shrek,Black Panther,Superman II
6,Jumanji,Hidden Figures,Star Trek IV: The Voyage Home
7,The Nightmare Before Christmas,Arrival,Star Trek III: The Search for Spock
8,The Mask,Star Wars: Episode VIII - The Last Jedi,Back to the Future Part III
9,Groundhog Day,Wonder Woman,The Dark Crystal


Here we clearly see that the recommendations for each user are different. This recommender could be used inside UnicornFlix on the main page under the title "Top picks for You", where each subscriber would be provided a list or titles that are personalized to their viewing habits, we will work on changing these in realtime later in this notebook.

If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export.

## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)

### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period, etc...). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [16]:
rerank_user = user
rerank_items = items_df.sample(15).index.tolist()
rerank_list = []
for item in rerank_items:
    movie = items_df.loc[item]['TITLE']
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Hurricane Streets
1,Only the Strong
2,Easy Rider
3,Iron Man 2
4,Party Monster
5,Du bei dao wang
6,A Fistful of Dollars
7,Lucas
8,Bossa Nova
9,Fatal Attraction


Now lets take that list and make the personalized ranking API call.

In [17]:
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = workshop_rerank_campaign_arn,
        userId = str(rerank_user),
        inputList = rerank_item_list,
        metadataColumns = {
                "ITEMS": ['TITLE']
        }
)

We will add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [18]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    try: 
        movie = item['metadata']['title']
    except:
        movie = items_df.loc[item['itemId']]['TITLE']
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Hurricane Streets,Easy Rider
1,Only the Strong,A Fistful of Dollars
2,Easy Rider,Du bei dao wang
3,Iron Man 2,Fatal Attraction
4,Party Monster,The Killers
5,Du bei dao wang,Hurricane Streets
6,A Fistful of Dollars,Devil in a Blue Dress
7,Lucas,Bossa Nova
8,Bossa Nova,Lucas
9,Fatal Attraction,Party Monster


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user that cannot be easily categorized in your metadata, for instance "Critics picks" which are curated by a person.

# Filters <a class="anchor" id="filters"></a>

## Create Filters <a class="anchor" id="create-filters"></a>
[Back to top](#top)

Personalize can utilize either [static or dynamic filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Static filters are where the filter properties are built into the filter itself, which makes invocation simpler, but gives less flexibility. An example of this would be a "Horror" movie filter, which invokes the get_recommendations_response api with the specific filter of GENRE = Horror. In order to create a recommendation for each filter that would require 10+ filters. Personalize also supports dynamic filters, where the values can be passed at runtime, allowing for a single filter of GENRE, where the actual genre is passed at runtime. 

Filters can be created for fields of both Items and Events. 

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Range based filters based on Item Metadata - Personalize supports range operations in both static and dynamic filters. Filtering based on a range can be used to create recommendations such as "Whats on now" in live tv scenarios, best of decade, Movies rated over 8/10 stars etc

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.


### Create the Genre filter

Within UnicornFlix, our catalog includes content from many different genres (Action, Documentary, Sci-Fi etc). Since there are a lot of genres to filter on, we will create a dynamic filter using the dynamic variable $GENRE, this will allow us to pass in the variable at runtime rather than create a static filter for each genre.

In [19]:
filter_name = 'Genre'
try:
    create_genre_filter_response = personalize.create_filter(
        name=filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.GENRES IN ($GENRE)'
    )
    
    genre_filter_arn = create_genre_filter_response['filterArn']

    print('Creating the Personalize filter with genre_filter_arn {}.'.format(genre_filter_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    genre_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+filter_name
    print('The Personalize filter {} already exists.'.format(filter_name))
    print ('\nWe will be using the existing Personalize Filter with genre_filter_arn = {}'.format(genre_filter_arn))
    

Creating the Personalize filter with genre_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/Genre.


### Create the Promotion filter

In [20]:
promotion_filter_name = 'promotion'
try:
    create_promotion_filter_response = personalize.create_filter(
        name=promotion_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.PROMOTION IN ("true")'
    )
    
    promotion_filter_arn = create_promotion_filter_response['filterArn']
    print('Creating the Personalize filter with promotion_filter_arn {}.'.format(promotion_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    promotion_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+promotion_filter_name
    print('The Personalize filter {} already exists.'.format(promotion_filter_name))
    print ('\nWe will be using the existing Personalize Filter with promotion_filter_arn = {}'.format(promotion_filter_arn))
    

Creating the Personalize filter with promotion_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/promotion.


### Create the Year Range filter
Personalize can also filter based on numerical ranges. This can be helpful if you want to look for items that are within a given time window, above a certain rating etc. For that we will create a filter for decades.

In [21]:
year_range_filter_name = 'YearRange'
try:
    create_genre_filter_response = personalize.create_filter(
        name=year_range_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.YEAR >= $YEAR1 AND Items.YEAR < $YEAR2'
    )
    
    year_range_filter_arn = create_genre_filter_response['filterArn']
    print('Creating the Personalize filter with year_range_filter_arn {}.'.format(year_range_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    year_range_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+year_range_filter_name
    print('The Personalize filter {} already exists.'.format(year_range_filter_name))
    print ('\nWe will be using the existing Personalize Filter with year_range_filter_arn = {}'.format(year_range_filter_arn))
    
    
    

Creating the Personalize filter with year_range_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/YearRange.


### Create filters for Watched and Unwatched movies
Lets also create 2 event filters for watched and unwatched content. The Top picks for you and More like X already have a filter on implemented to filter out watched events.

In [22]:
watched_filter_name = 'watched'

try:
    create_watched_filter_response = personalize.create_filter(
        name=watched_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Interactions.event_type IN ("Watch")'
    )
    
    watched_filter_arn = create_watched_filter_response['filterArn']
    print('Creating the Personalize filter with watched_filter_arn {}.'.format(watched_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    watched_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+watched_filter_name
    print('The Personalize filter {} already exists.'.format(watched_filter_name))
    print ('\nWe will be using the existing Personalize Filter with watched_filter_arn = {}'.format(watched_filter_arn))
    

unwatched_filter_name = 'unwatched'
try:
    create_unwatched_filter_response = personalize.create_filter(
        name = unwatched_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'EXCLUDE ItemID WHERE Interactions.event_type IN ("Watch")'
    )
    unwatched_filter_arn = create_unwatched_filter_response['filterArn']
    print('Creating the Personalize filter with unwatched_filter_arn {}.'.format(unwatched_filter_arn))

except personalize.exceptions.ResourceAlreadyExistsException as e:
    unwatched_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+unwatched_filter_name
    print('\nThe Personalize filter {} already exists.'.format(unwatched_filter_name))
    print ('\nWe will be using the existing Personalize Filter with unwatched_filter_arn = {}'.format(unwatched_filter_arn))
    

Creating the Personalize filter with watched_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/watched.
Creating the Personalize filter with unwatched_filter_arn arn:aws:personalize:us-east-1:381491864570:filter/unwatched.


Before we move on we want to add those filters to a list as well so they can be used later.

In [23]:
interaction_filter_arns = [watched_filter_arn, unwatched_filter_arn]

In [24]:
interaction_filter_arns

['arn:aws:personalize:us-east-1:381491864570:filter/watched',
 'arn:aws:personalize:us-east-1:381491864570:filter/unwatched']

In [25]:
max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    # Genre filter
    version_response = personalize.describe_filter(
        filterArn = genre_filter_arn
    )
    status_genre_filter = version_response["filter"]["status"]

    if status_genre_filter == "ACTIVE":
        print("Build succeeded for {}".format(genre_filter_arn))
        
    elif status_genre_filter == "CREATE FAILED":
        print("Build failed for {}".format(genre_filter_arn))
        break
        
    if not status_genre_filter == "ACTIVE":
        print("The filter Genre is still in progress")
    else:
        print("The filter Genre is ACTIVE")

    # Promotion filter
    version_response = personalize.describe_filter(
        filterArn = promotion_filter_arn
    )
    status_promotion_filter = version_response["filter"]["status"]

    if status_promotion_filter == "ACTIVE":
        print("Build succeeded for {}".format(promotion_filter_arn))
        
    elif status_promotion_filter == "CREATE FAILED":
        print("Build failed for {}".format(promotion_filter_arn))
        break
        
    if not status_promotion_filter == "ACTIVE":
        print("The filter Promotion is still in progress")
    else:
        print("The filter Promotion is ACTIVE")   
        
        
    # Decade filter
    version_response = personalize.describe_filter(
        filterArn = year_range_filter_arn
    )
    status_year_range_filter = version_response["filter"]["status"]

    if status_year_range_filter == "ACTIVE":
        print("Build succeeded for {}".format(year_range_filter_arn))
        
    elif status_year_range_filter == "CREATE FAILED":
        print("Build failed for {}".format(year_range_filter_arn))
        break
        
    if not status_year_range_filter == "ACTIVE":
        print("The filter YearRange is still in progress")
    else:
        print("The filter YearRange is ACTIVE")

    # Watched filter
    version_response = personalize.describe_filter(
        filterArn = watched_filter_arn
    )
    status_watched_filter = version_response["filter"]["status"]

    if status_watched_filter == "ACTIVE":
        print("Build succeeded for {}".format(watched_filter_arn))
        
    elif status_watched_filter == "CREATE FAILED":
        print("Build failed for {}".format(watched_filter_arn))
        break
        
    if not status_watched_filter == "ACTIVE":
        print("The filter Watched is still in progress")
    else:
        print("The filter Watched is ACTIVE")
 
    # Unatched filter
    version_response = personalize.describe_filter(
        filterArn = unwatched_filter_arn
    )
    status_unwatched_filter = version_response["filter"]["status"]

    if status_unwatched_filter == "ACTIVE":
        print("Build succeeded for {}".format(unwatched_filter_arn))
        
    elif status_unwatched_filter == "CREATE FAILED":
        print("Build failed for {}".format(unwatched_filter_arn))
        break
        
    if not status_unwatched_filter == "ACTIVE":
        print("The filter Unwatched is still in progress")
    else:
        print("The filter Unwatched is ACTIVE")
    
    if status_genre_filter == "ACTIVE" and status_year_range_filter == 'ACTIVE' and status_promotion_filter  == 'ACTIVE' and status_watched_filter == "ACTIVE" and status_unwatched_filter == "ACTIVE":
        break

    print()
    time.sleep(30)

The filter Genre is still in progress
The filter Promotion is still in progress
The filter YearRange is still in progress
The filter Watched is still in progress
The filter Unwatched is still in progress

Build succeeded for arn:aws:personalize:us-east-1:381491864570:filter/Genre
The filter Genre is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:381491864570:filter/promotion
The filter Promotion is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:381491864570:filter/YearRange
The filter YearRange is ACTIVE
The filter Watched is still in progress
The filter Unwatched is still in progress

Build succeeded for arn:aws:personalize:us-east-1:381491864570:filter/Genre
The filter Genre is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:381491864570:filter/promotion
The filter Promotion is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:381491864570:filter/YearRange
The filter YearRange is ACTIVE
The filter Watched is still in progress
The filter Unwatched i

## Using Filters <a class="anchor" id="using-filters"></a>
[Back to top](#top)

Now that the Filters have been created we can use them to filter our recommendations.

### Get recommendations using a static filter

Earlier we chose to promote a specific set of movies (movies set in or about Las Vegas), which we created the item metadata field of "PROMOTION", and set it to true for 35 movies. We will use a static filter to ensure that these items are retured in recommendations. First we will use a static filter to ensure that ALL recommendations meet this criteria of the filter.

In [26]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_w_promotions_only_df_users(recommendations_df, user_id):

    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = promotion_filter_arn,
        numResults = 15,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        movie = item['metadata']['title']
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [27]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_w_promotions_only_df_users(recommendations_df_users, user)

recommendations_df_users

,278,583,479
0,The Hangover,Next,Diamonds Are Forever
1,Ocean's Eleven,Last Vegas,Vegas Vacation
2,Fear and Loathing in Las Vegas,Paul Blart: Mall Cop 2,Race to Witch Mountain
3,Dodgeball,Dodgeball,Rat Race
4,Smokin' Aces,The Hangover,Next
5,Leaving Las Vegas,The Hangover Part II,Miss Congeniality 2: Armed & Fabulous
6,Bugsy,Wild Card,Showgirls
7,Diamonds Are Forever,Smokin' Aces,The Gauntlet
8,The Hangover Part II,Ocean's Eleven,Ocean's 11
9,Rat Race,Rat Race,Dodgeball


In this scenario for the 3 test users we got recommendations for previously, the static filter limits the recommendations to only those that meet the filter criteria. This does not provide a good balance of recommendations that are relevant and recommendations that meet the filter criteria. We could use this inside UnicornFlix under a carousel/rail named "Vegas Movies", as it is only providing recommendations for Vegas movies.

### Get recommendations using a promotions filter

Personalize has a capability to "promote" certain items, which can be defined as a promotion filter. A configurable subset of the recommendations returned will be from the items defined as promotion, in this case 30%. This should provide a balance of items that are the most relevant, and items from the promoted items. Note: The promoted items will be the most relevant items in the promoted items subset based on that users interactions.

In [28]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_w_promotions_df_users(recommendations_df, user_id):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = 15,
        promotions = [{
        "name" : "vegas_promotion",
        "percentPromotedItems" : 30,
        "filterArn": promotion_filter_arn,
        }],
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    promotion_list = []
    recommendation_list = []
    for item in itemList:
        try:
            movie = item['metadata']['title']
        except:
            movie = items_df.loc[item['itemId']]['TITLE']
        try: 
            item["promotionName"]
            promotion_list.append('Y')
        except KeyError:
            promotion_list.append('N')
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    promotions_df = pd.DataFrame(promotion_list, columns = [user_id])
    promotions_df.columns = ['Promotion']
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    recommendations_df = pd.concat([recommendations_df, promotions_df], axis=1)
    return recommendations_df

In [29]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_w_promotions_df_users(recommendations_df_users, user)

recommendations_df_users

,278,Promotion,583,Promotion,479,Promotion
0,Up,N,Interstellar,N,Star Trek: The Motion Picture,N
1,Apollo 13,N,Next,Y,Diamonds Are Forever,Y
2,Harry Potter and the Half-Blood Prince,N,Ant-Man,N,Moonraker,N
3,The Hangover,Y,Last Vegas,Y,Conan the Destroyer,N
4,Ocean's Eleven,Y,The Imitation Game,N,The Karate Kid Part II,N
5,The Incredibles,N,The Martian,N,Vegas Vacation,Y
6,Fear and Loathing in Las Vegas,Y,Guardians of the Galaxy,N,Star Trek V: The Final Frontier,N
7,Toy Story,N,Black Panther,N,Superman II,N
8,Shrek,N,Hidden Figures,N,Race to Witch Mountain,Y
9,Dodgeball,Y,Arrival,N,Star Trek IV: The Voyage Home,N


We can see that promotions are mixed in with the most relevant items for the users. This is different from applying a filter directly where **all** items recommended would belong to the promotion. If a user already had certain promoted items in their recommendations before apllying the promotions, they will still be recommended. Inside UnicornFlix, we could use this as the main page "Top picks for you" carousel/rail, where the recommendations are across all of the catalog titles, with Vegas movies being inserted into the recommndations.

### Get recommendations using the date range filter

First lets create functions to get recommendations and pass in dynamic filter values.


In [30]:
def get_new_recommendations_df_by_year_range_filter(recommendations_df, user_id, year_range_filter_arn, filter_value1, filter_value2):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = year_range_filter_arn,
        filterValues = {"YEAR1": filter_value1,"YEAR2": filter_value2},
        numResults = 15,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = item['metadata']['title']
        recommendation_list.append(movie)
    filter_name = year_range_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_value1])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [31]:
decades_to_filter = [1930,1940,1950,1960,1970,1980,1990,2000,2010]

In [32]:
# Iterate through Decades
recommendations_df_decade_shelves = pd.DataFrame()
for decade in decades_to_filter:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_year_range_filter(recommendations_df_decade_shelves, user, year_range_filter_arn , str(decade), str(decade+10))
    
recommendations_df_decade_shelves

,1930,1940,1950,1960,1970,1980,1990,2000,2010
0,King Kong,Flash Gordon Conquers the Universe,Attack of the Puppet People,Barbarella,Star Trek: The Motion Picture,Conan the Destroyer,Back to the Future Part III,Star Wars: Episode II - Attack of the Clones,Alice in Wonderland
1,Alice in Wonderland,Frankenstein Meets the Wolf Man,The Atomic Submarine,The Dirty Dozen,Moonraker,The Karate Kid Part II,Star Trek: Insurrection,Scooby-Doo,The Sorcerer's Apprentice
2,The Mummy,Dragon Seed,The Blob,You Only Live Twice,Superman,Star Trek V: The Final Frontier,Star Wars: Episode I - The Phantom Menace,Mission to Mars,Oz the Great and Powerful
3,The Bride of Frankenstein,The Third Man,Attack of the Crab Monsters,From Russia with Love,Jaws 2,Superman II,Arachnophobia,Star Trek: Nemesis,Journey 2: The Mysterious Island
4,All Quiet on the Western Front,Citizen Kane,The Thing from Another World,On Her Majesty's Secret Service,Conquest of the Planet of the Apes,Star Trek IV: The Voyage Home,Teenage Mutant Ninja Turtles,Spy Kids,The Chronicles of Narnia: The Voyage of the Da...
5,Frankenstein,Suspicion,20 Million Miles to Earth,Thunderball,The Black Hole,Star Trek III: The Search for Spock,The Witches,Spy Kids 2: Island of Lost Dreams,Gulliver's Travels
6,Jamaica Inn,The Maltese Falcon,This Island Earth,Bullitt,Logan's Run,The Dark Crystal,DragonHeart,City of Ember,Minions
7,The Adventures of Robin Hood,The Wolf Man,Alice in Wonderland,Dr. No,The Spy Who Loved Me,Masters of the Universe,The Flintstones,The League of Extraordinary Gentlemen,Season of the Witch
8,Duck Soup,She Wore a Yellow Ribbon,The Bridge on the River Kwai,Chitty Chitty Bang Bang,Diamonds Are Forever,"Honey, I Shrunk the Kids",Congo,Final Fantasy: The Spirits Within,Suicide Squad
9,Dracula,The Grapes of Wrath,To Catch a Thief,King Kong vs. Godzilla,A Bridge Too Far,Time Bandits,Predator 2,The Haunted Mansion,The BFG


We can see that creating just one filter, we are able to get a variety of subsets of movies that can be used to populate multiple carousels/rails in UnicornFlix ("1990s movies", "1980s movies" etc)

### Get recommendations unsing the genre filter

Now that we have the ability to generate recommendations via the "Top Picks For You" Recommender, we would like to extend this feature to all of the rails/shleves in the UnicornFlix app. We can use a dynamic filter to provide recommendations based on any item metadata field. In this case we will use the "GENRES" categorical information in the IMDb dataset. 

In [33]:
def get_new_recommendations_df_by_genre_filter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""},
        numResults = 15,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        try:
            movie = item['metadata']['title']
        except:
            movie = items_df.loc[item['itemId']]['TITLE']
        recommendation_list.append(movie)
    filter_name = genre_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [34]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = items_df
# Render some sample data
items_meta_df.head(5)

,TITLE,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,PLOT,US_MATURITY_RATING_STRING,US_MATURITY_RATING,GENRES,CREATION_TIMESTAMP,PROMOTION
ITEM_ID,,,,,,,,,,
tt0906325,Bagboy,2007,4,741,A teenager enters the competitive world of gro...,PG-13,13,Comedy,1167609600,False
tt0906665,Sukiyaki Western Django,2007,6,14582,A nameless gunfighter arrives in a town ripped...,R,17,Action|Western,1167609600,False
tt0907657,Once,2007,8,109787,A modern-day musical about a busker and an imm...,R,17,Drama|Music|Romance,1167609600,False
tt0910905,In the Electric Mist,2009,6,16592,A detective in post-Katrina New Orleans has a ...,R,17,Crime|Drama|Mystery|Thriller,1230768000,False
tt0910554,Frequently Asked Questions About Time Travel,2009,7,32051,"While drinking at their local pub, three socia...",PG-13,13,Comedy|Sci-Fi,1230768000,False


Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [35]:
unique_genre_field_values = items_meta_df['GENRES'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [36]:
genres_to_filter

['Short',
 'Sci-Fi',
 'Biography',
 'Family',
 'Sport',
 'Romance',
 'Fantasy',
 'Documentary',
 'War',
 'History',
 'Film-Noir',
 'Music',
 'Musical',
 'Crime',
 'Drama',
 'News',
 'Horror',
 'Mystery',
 'Western',
 'Comedy',
 'Thriller',
 'Action',
 'Animation',
 'Adventure']

In [37]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_genre_filter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

,Short,Sci-Fi,Biography,Family,Sport,Romance,Fantasy,Documentary,War,History,...,Drama,News,Horror,Mystery,Western,Comedy,Thriller,Action,Animation,Adventure
0,A Trip to the Moon,Star Trek: The Motion Picture,Out of Africa,The Karate Kid Part II,The Karate Kid Part II,Splash,Conan the Destroyer,Roger & Me,A Bridge Too Far,A Bridge Too Far,...,WarGames,An Inconvenient Truth,Arachnophobia,Star Trek: The Motion Picture,Back to the Future Part III,Star Trek IV: The Voyage Home,Moonraker,Moonraker,The Lord of the Rings,Star Trek: The Motion Picture
1,A Cosmic Christmas,Moonraker,Patton,The Dark Crystal,The Karate Kid,Romancing the Stone,Star Trek V: The Final Frontier,It Came from Hollywood,The Dirty Dozen,The Mission,...,The NeverEnding Story,"Food, Inc.",Jaws 2,2010,Three Amigos!,Back to the Future Part III,Star Trek V: The Final Frontier,Conan the Destroyer,Space Jam,Moonraker
2,Porky in Wackyland,Star Trek V: The Final Frontier,Glory,"Honey, I Shrunk the Kids",The Karate Kid Part III,Howard the Duck,The Dark Crystal,Super Size Me,1941,The Man Who Would Be King,...,Superman,Blackfish,The Witches,The Witches,Westworld,"Honey, I Shrunk the Kids",Never Say Never Again,The Karate Kid Part II,The Transformers: The Movie,Conan the Destroyer
3,The Prince and the Pauper,Superman II,Chariots of Fire,Clash of the Titans,Days of Thunder,Legend,Masters of the Universe,Pumping Iron,MASH,Empire of the Sun,...,The Karate Kid,Born Into Brothels: Calcutta's Red Light Kids,Invasion of the Body Snatchers,Congo,Maverick,Time Bandits,2010,Star Trek V: The Final Frontier,The Black Cauldron,Star Trek V: The Final Frontier
4,Porky's Hare Hunt,Star Trek IV: The Voyage Home,Butch Cassidy and the Sundance Kid,Flight of the Navigator,Space Jam,Willow,Time Bandits,An Inconvenient Truth,Biggles: Adventures in Time,Gallipoli,...,The Karate Kid Part III,Taxi to the Dark Side,Predator 2,Star Trek: Generations,Young Guns II,Airplane II: The Sequel,Star Trek: Insurrection,Superman II,Nausicaä of the Valley of the Wind,Superman II
5,Runaway Brain,Star Trek III: The Search for Spock,Lawrence of Arabia,The NeverEnding Story,Over the Top,The Three Musketeers,Clash of the Titans,Journeys with George,The Man Who Would Be King,Glory,...,A Bridge Too Far,Citizenfour,Gremlins 2: The New Batch,Scooby-Doo,Young Guns,Superman III,WarGames,Star Trek III: The Search for Spock,James and the Giant Peach,Star Trek IV: The Voyage Home
6,How the Grinch Stole Christmas!,Back to the Future Part III,Born on the Fourth of July,Teenage Mutant Ninja Turtles,Rocky IV,The Jewel of the Nile,Splash,Blind Spot. Hitler's Secretary,Empire of the Sun,The Bounty,...,Willow,Burma VJ: Reporting from a Closed Country,King Kong,Return to Oz,Alvarez Kelly,Splash,Octopussy,Masters of the Universe,Final Fantasy: The Spirits Within,Star Trek III: The Search for Spock
7,Dragon Ball Z: Dead Zone,Masters of the Universe,The Last Emperor,The Karate Kid,Rocky II,Phenomenon,Star Wars: Episode I - The Phantom Menace,Theremin: An Electronic Odyssey,Patton,Cool Runnings,...,The NeverEnding Story II: The Next Chapter,13th,Poltergeist,The Golden Child,Dances with Wolves,Flight of the Navigator,Arachnophobia,Never Say Never Again,The Great Mouse Detective,Back to the Future Part III
8,Luxo Jr.,"Honey, I Shrunk the Kids","Good Night, and Good Luck.",The Karate Kid Part III,Rocky III,Starman,The NeverEnding Story,Antonio Gaudí,Gallipoli,Royal Flash,...,The Mission,Restrepo,A Nightmare on Elm Street 5: The Dream Child,Clue,Pale Rider,Arachnophobia,Innerspace,Star Trek: Insurrection,Dragon Ball: Curse of the Blood Rubies,The Dark Crystal
9,The Great Train Robbery,Time Bandits,"Good Morning, Vietnam",Short Circuit,Rocky V,Bill & Ted's Excellent Adventure,Arachnophobia,The War Room,Farewell to the King,Lawrence of Arabia,...,Red Dawn,The Culture High,Return to Oz,Soylent Green,Wild Wild West,Innerspace,Jaws 2,Clash of the Titans,Wizards,Masters of the Universe


Great! Now we have a personalized list of recommendations for each genre in our catalog. We can use this on the homepage for each of the carousels/rails that correspond to genre, such as "Action movies", "Documentaries" etc.

## Real-time Events<a class="anchor" id="real-time"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

In [38]:
event_tracker_name = 'MovieTracker'
try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    tracking_id = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(tracking_id))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    tracking_id = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd trackingId = {}'.format(tracking_id))

{
  "eventTrackerArn": "arn:aws:personalize:us-east-1:381491864570:event-tracker/8a121744",
  "trackingId": "b55b7fbb-cf0a-4e82-9ee3-e71a27ef2c2b",
  "ResponseMetadata": {
    "RequestId": "003ab2e0-5339-41dc-acde-76c7ddfece7d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 27 Apr 2024 23:31:46 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "139",
      "connection": "keep-alive",
      "x-amzn-requestid": "003ab2e0-5339-41dc-acde-76c7ddfece7d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}

Creating the event_tracker with event_tracker_arn = arn:aws:personalize:us-east-1:381491864570:event-tracker/8a121744

And trackingId = b55b7fbb-cf0a-4e82-9ee3-e71a27ef2c2b


In [39]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:

    version_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = version_response['eventTracker']['status']

    if status == 'ACTIVE':
        print('Build succeeded for {}'.format(event_tracker_arn))
    elif status == "CREATE FAILED":
        print('Build failed for {}'.format(wevent_tracker_arn))
        in_progress_campaigns.remove(event_tracker_arn)
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
    else:
        print('The event tracker build is still in progress')
        
    time.sleep(15)

The event tracker build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:381491864570:event-tracker/8a121744


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [40]:
sessionDict = {}

def send_movie_click(user_id, item_id, event_type):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    session_dict = {}
    try:
        session_id = sessionDict[str(user_id)]
    except:
        session_dict[str(user_id)] = str(uuid.uuid1())
        session_id = session_dict[str(user_id)]
        
    # Configure Properties:
    event = {
    "itemId": str(item_id),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = tracking_id,
    userId= str(user_id),
    sessionId = session_id,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(event_type),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = items_df.loc[item_id]['TITLE']
    
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + movie_name)
    send_movie_click(user_id=user_id, item_id=item_id,event_type=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    time.sleep(2)
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = 15,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        try:
            movie = item['metadata']['title']
        except:
            movie = items_df.loc[item['itemId']]['TITLE']
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [41]:
# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(rerank_user),
        numResults = 15,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    try:
        movie = item['metadata']['title']
    except:
        movie = items_df.loc[item['itemId']]['TITLE']
    recommendation_list.append(movie)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [rerank_user])
user_recommendations_df

,479
0,Star Trek: The Motion Picture
1,Moonraker
2,Conan the Destroyer
3,The Karate Kid Part II
4,Star Trek V: The Final Frontier
5,Superman II
6,Star Trek IV: The Voyage Home
7,Star Trek III: The Search for Spock
8,Back to the Future Part III
9,The Dark Crystal


Now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [42]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [43]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, rerank_user, movie,'click')
    time.sleep(5)
    

sending event click for High Crimes
sending event click for Dead of Night
sending event click for Matinee


Now we can look at how the click events changed the recommendations. Note: given that only a single movie was clicked between each get recomendations call, the changes may be subtle. The more interactions a user sends in the larger the change in recommendations will be.

In [44]:
user_recommendations_df

,479,High Crimes,Dead of Night,Matinee
0,Star Trek: The Motion Picture,The General's Daughter,Butch Cassidy and the Sundance Kid,Searching for Bobby Fischer
1,Moonraker,Enough,Citizen Kane,The Madness of King George
2,Conan the Destroyer,The Net,Anatomy of a Murder,Mr. Holland's Opus
3,The Karate Kid Part II,High Crimes,Lawrence of Arabia,Eight Men Out
4,Star Trek V: The Final Frontier,Gone in 60 Seconds,The Bridge on the River Kwai,Chariots of Fire
5,Superman II,House on Haunted Hill,To Kill a Mockingbird,Wyatt Earp
6,Star Trek IV: The Voyage Home,Murder by Numbers,Casablanca,Little Women
7,Star Trek III: The Search for Spock,Soylent Green,Dracula,A Bridge Too Far
8,Back to the Future Part III,Dick Tracy,Contact,The Client
9,The Dark Crystal,Driven,The Bride of Frankenstein,"Corrina, Corrina"


In the cell above, the first column after the index is the user's default recommendations from the "Top picks for you" recommender, and each column after that has as a header of the movie that they interacted with via a real time event, and the recommendations after this event occurred. 

Inside of UnicornFlix, we would use this feature to make sure that the list of recommendations are staying current with exactly what the viewer is interacting with the application.

The behavior may not shift very much or a lot; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies to see the impact.

### Filtering on user's recent interactions
Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) 

We will create a new helper function to use the personalized ranking campaign, sice the recommenders have an inbuilt filter to filter out watched content.

In [45]:
def get_new_ranked_recommendations_df_by_static_filter(recommendations_df, user_id, rerank_item_list, filter_arn):
    
    # Todo: update with new feature     
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_personalized_ranking(
        campaignArn = workshop_rerank_campaign_arn,
        userId = str(user_id),
        inputList = rerank_item_list,
        filterArn = filter_arn,
        metadataColumns = {
            "ITEMS": ['TITLE']
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['personalizedRanking']
    recommendation_list = []
    for item in item_list:
        try:
            movie = item['metadata']['title']
        except:
            movie = items_df.loc[item['itemId']]['TITLE']
        recommendation_list.append(movie)


    filter_name = filter_arn.split('/')[1]
    new_rec_df = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [46]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_ranked_recommendations_df_by_static_filter(recommendations_df_events, rerank_user, rerank_item_list, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,NaN,Devil in a Blue Dress
1,NaN,Easy Rider
2,NaN,The Killers
3,NaN,Fatal Attraction
4,NaN,Only the Strong
5,NaN,A Fistful of Dollars
6,NaN,Lucas
7,NaN,Du bei dao wang
8,NaN,The Thief of Bagdad
9,NaN,Bossa Nova


It is very likely that the `watched` column of the dataframe will have several `NaN` values, this is because we randomly selected some movies from the catalog to rank and our randomly selected user has not watched them.

Now lets send a watch event in for the top 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [47]:
ranked_unwatched_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    userId = str(rerank_user),
    inputList = rerank_item_list,
    filterArn = filter_arn,
    metadataColumns = {
        "ITEMS": ['TITLE']
    })

item_list = ranked_unwatched_recommendations_response['personalizedRanking'][:4]

# Todo: update with new feature  

for item in item_list:
    print('sending event Watch for ' + items_df.loc[item['itemId']]['TITLE'])
    send_movie_click(user_id=rerank_user, item_id=item['itemId'], event_type='Watch')
    time.sleep(5)

sending event Watch for Devil in a Blue Dress
sending event Watch for Easy Rider
sending event Watch for The Killers
sending event Watch for Fatal Attraction


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [48]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_ranked_recommendations_df_by_static_filter(recommendations_df_events, rerank_user, rerank_item_list, filter_arn)
recommendations_df_events

,watched,unwatched
0,Fatal Attraction,Lucas
1,Easy Rider,Bossa Nova
2,Devil in a Blue Dress,A Fistful of Dollars
3,The Killers,Hurricane Streets
4,NaN,Du bei dao wang
5,NaN,Underworld
6,NaN,Party Monster
7,NaN,Only the Strong
8,NaN,The Thief of Bagdad
9,NaN,Jackass 3.5


Inside of UnicornFlix we could use this feature to filter out movies that have already been watched, so that recommendations will be for movies that the viewer has not already seen. Another feature that can be developed is a "Watch again" carousel/rail, where you can recommend content that has been seen in the past, but due to current viewing habits, may be interesting to the viewer to revisit.

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`Media_05_Clean_Up.ipynb`](Media_05_Clean_Up.ipynb).

In [49]:
%store workshop_dataset_group_arn
%store region
%store role_name

Stored 'workshop_dataset_group_arn' (str)
Stored 'region' (str)
Stored 'role_name' (str)
